# Гомогенность и полнота (Homogeneity and completeness)
Кластерный анализ - это метод машинного обучения, который используется для группировки похожих объектов в кластеры. При этом возникает необходимость оценки качества полученных кластеров.

Гомогенность - это мера того, насколько все объекты внутри одного кластера похожи между собой. Если все объекты в кластере очень похожи друг на друга, то гомогенность высокая.

Полнота - это мера того, насколько объекты одного класса распределены между кластерами. Если все объекты одного класса находятся в одном кластере, то полнота высокая.

То есть, гомогенность и полнота - это две меры, которые помогают понять, насколько полученные кластеры содержат объекты одного класса и насколько все объекты внутри кластера похожи между собой. Эти меры помогают оценить качество кластеризации.

Давай представим, что у нас есть набор фотографий животных, и мы пытаемся разделить их на две категории: котики и собачки. Для этого мы используем алгоритм кластеризации и получаем два кластера: котиков и собачек.

Теперь давай представим, что у нас есть набор правильных меток для каждой фотографии, которые говорят нам, является ли животное на фото котиком или собачкой. Например, мы можем использовать набор меток, созданный экспертом, который разметил каждую фотографию как котика или собачку.

Гомогенность и полнота могут помочь нам оценить, насколько хорошо наш алгоритм кластеризации справился с задачей разделения на категории. Гомогенность измеряет, насколько много объектов из одной категории было помещено в один кластер. Например, если в одном кластере окажется большинство котиков, а в другом большинство собачек, то гомогенность будет высокой. Полнота измеряет, насколько много объектов из одной категории было найдено и помещено в соответствующий кластер. Например, если все котики были правильно размещены в кластере котиков, а собачки в кластере собачек, то полнота будет высокой.

In [1]:
import numpy as np
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

from sklearn.metrics import homogeneity_score, completeness_score
from sklearn.metrics.cluster import contingency_matrix

In [2]:
# создадим два вектора
# с истинными классами
labels_true = [1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 4]
# и результатом кластеризации
labels_pred = [1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4]

In [3]:
# создадим таблицу сопряженности, где строки — это классы, столбцы — кластеризация
cont_matrix = contingency_matrix(labels_true, labels_pred)
# найдем общее количество наблюдений
n = len(labels_true)
# создадим вектор с суммой по каждому классу
sum_rows = np.sum(cont_matrix, axis=1)
# создадим вектор с суммой по каждому кластеру
sum_cols = np.sum(cont_matrix, axis=0)

### Найдем гомогенность
Когда все наблюдения в кластере $k$ имеют одинаковую метку $c$, гомогенность равна $1$. Можно сказать, что гомогенность — это когда каждый кластер содержит только наблюдения одного класса. $h \in [0 , 1]$

$ h = 1 - \frac{H(C|K)}{H(C)} \\
H(C|K) = \sum_{c=1}^C \sum_{k=1}^K \frac{n_{c,k}}{n} log \Big( \frac{n_{c,k}}{n_k} \Big) \\
H(C) = \sum_{c=1}^C \frac{n_{c}}{n} log \Big( \frac{n_{c}}{n} \Big) $

, где $n$ — общее количество наблюдений, $c$ – количество классов, $k$ – количество кластеров, $n_c$ — наблюдений в классе $с$, $n_k$ — наблюдений в кластере $k$, $n_{c, k}$ — наблюдений с меткой класса $с$ в кластере $k$


In [4]:
# реализуем формулу поиска гомогенности
H_CK = 0
# извлечем строки таблицы сопряженности,
# они содержат принадлежность наблюдений к классу
for cols in cont_matrix:
    # извлечем количество наблюдений с меткой c в классе k
    # извлечем число наблюдений в каждом кластере
    for n_ck, n_c in zip(cols, sum_cols):
        if n_ck > 0:
            # реализуем формулу H(C|K)
            H_CK -= n_ck/n * np.log(n_ck/n_c)

H_C = 0
# извлечем количество наблюдений в каждом классе
for n_c in sum_rows:
    # реализуем формулу H(C)
    H_C -= n_c/n * np.log(n_c/n)

h = 1 - (H_CK/H_C)

In [5]:
# компактное представление и проверка результата с помощью sklearn
H_CK = - np.sum(np.sum([[n_ck/n * np.log(n_ck/n_k) for n_ck, n_k in zip(cols, sum_cols) if n_ck > 0] for cols in cont_matrix]))
H_C = - np.sum([n_c/n * np.log(n_c/n) for n_c in sum_rows])
h = 1 - (H_CK/H_C)
print('H_CK: ', H_CK)
print('H_C: ', H_C)
print('h: ', h)
print('h_score: ', homogeneity_score(labels_true, labels_pred))

H_CK:  0.37808028030542473
H_C:  1.3730039128771998
h:  0.7246327728861761
h_score:  0.7246327728861762


### Найдем полноту
Можно сказать, что полнота — это зеркальная мера гомогенности. Полнота равна $1$, когда все наблюдения класса $с$ всегда оказываются в одном кластере $k$. $c \in [0 , 1]$

$ c = 1 - \frac{H(K|C)}{H(K)} \\
H(K|C) = \sum_{k=1}^K \sum_{c=1}^C \frac{n_{k,c}}{n} log \Big( \frac{n_{k,c}}{n_c} \Big) \\
H(K) = \sum_{k=1}^K \frac{n_{k}}{n} log \Big( \frac{n_{k}}{n} \Big) $

, где $n$ — общее количество наблюдений, $c$ – количество классов, $k$ – количество кластеров, $n_c$ — наблюдений в классе $с$, $n_k$ — наблюдений в кластере $k$, $n_{k, c}$ — наблюдений в кластере $k$ с меткой класса $с$

In [6]:
# реализуем формулу поиска полноты
H_KC = 0
# извлечем строки таблицы сопряженности,
# они содержат принадлежность наблюдений к классу
# извлечем число наблюдений в каждом классе
for cols, n_c in zip(cont_matrix, sum_rows):
    # извлечем количество наблюдений в классе k с меткой c
    for n_kc in cols:
        if n_kc > 0:
            # реализуем формулу H(K|C)
            H_KC -= n_kc/n * np.log(n_kc/n_c)

H_K = 0
# извлечем количество наблюдений в каждом кластере
for n_k in sum_cols:
    # реализуем формулу H(K)
    H_K -= n_k/n * np.log(n_k/n)

In [7]:
# компактное представление и проверка результата с помощью sklearn
H_K = - np.sum([n_k/n * np.log(n_k/n) for n_k in sum_cols])
H_KC = - np.sum(np.sum([[n_kc/n * np.log(n_kc/n_c) for n_kc in cols if n_kc > 0] for cols, n_c in zip(cont_matrix, sum_rows)]))
c = 1 - (H_KC/H_K)
print('H_K:', H_K)
print('H_KC:', H_KC)
print('c:', c)
print('c_score:', completeness_score(labels_true, labels_pred))

H_K: 1.3421131789144003
H_KC: 0.3471895463426252
c: 0.7413112755338134
c_score: 0.7413112755338134
